Two main types
*   *instruction fine-tuning*
*   *classification fine-tuning*




**Preparing dataset**

In [1]:
import urllib.request
import zipfile
import os
from pathlib import Path
url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"
def download_and_unzip_spam_data(
 url, zip_path, extracted_path, data_file_path):
 if data_file_path.exists():
  print(f"{data_file_path} already exists. Skipping download "
  "and extraction."
  )
  return

with urllib.request.urlopen(url) as response: #Download File
 with open(zip_path, "wb") as out_file:
  out_file.write(response.read())

 with zipfile.ZipFile(zip_path, "r") as zip_ref: # Unzips the file
  zip_ref.extractall(extracted_path)

 original_file_path = Path(extracted_path) / "SMSSpamCollection"
 os.rename(original_file_path, data_file_path)  #Adds a .tsv extension
 print(f"File downloaded and saved as {data_file_path}")

download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)

File downloaded and saved as sms_spam_collection/SMSSpamCollection.tsv
sms_spam_collection/SMSSpamCollection.tsv already exists. Skipping download and extraction.


Loading dataset into pandas dataframe

In [2]:
import pandas as pd
df = pd.read_csv(
 data_file_path, sep="\t", header=None, names=["Label", "Text"]
)

print(df)

     Label                                               Text
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham               Will ü b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


**Class Label distribution**

In [3]:
print(df["Label"].value_counts())

Label
ham     4825
spam     747
Name: count, dtype: int64


Creating a balanced dataset

In [4]:
def create_balanced_dataset(df):
 num_spam = df[df["Label"] == "spam"].shape[0] # Counts spam instances
 ham_subset = df[df["Label"] == "ham"].sample(
 num_spam, random_state=123
 )

 balanced_df = pd.concat([
 ham_subset, df[df["Label"] == "spam"]
 ])
 return balanced_df

balanced_df = create_balanced_dataset(df)
print(balanced_df["Label"].value_counts())

Label
ham     747
spam    747
Name: count, dtype: int64


Converting string class labels into integer class labels (0 & 1)



In [5]:
balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})

Splitting dataset into Training, validation and testing

In [6]:
def random_split(df, train_frac, validation_frac):

 df = df.sample(
 frac=1, random_state=123
 ).reset_index(drop=True) #Shuffles the entire dataframe
 train_end = int(len(df) * train_frac) #Calculate split indices
 validation_end = train_end + int(len(df) * validation_frac)

 #Split dataframe
 train_df = df[:train_end]
 validation_df = df[train_end:validation_end]
 test_df = df[validation_end:]
 return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(
 balanced_df, 0.7, 0.1)

Save dataset as csv file

In [7]:
train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)

**Creating Dataloaders**

Pad the shortest message to the length of longest message

In [8]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.2 MB/s eta 0:00:00


In [9]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


Will use <|endoftext|> for padding

Define dataset class

In [10]:
import torch
from torch.utils.data import Dataset

import pandas as pd

In [11]:
class SpamDataset(Dataset):
 def __init__(self, csv_file, tokenizer, max_length=None,
 pad_token_id=50256):
  self.data = pd.read_csv(csv_file)

 #Pretokenizes text
  self.encoded_texts = [
  tokenizer.encode(text) for text in self.data["Text"]
  ]

  if max_length is None:
    self.max_length = self._longest_encoded_length()
  else:
    self.max_length = max_length

  #Truncates sequences if they are longer than max length
  self.encoded_texts = [
  encoded_text[:self.max_length]
  for encoded_text in self.encoded_texts
  ]

  self.encoded_texts = [
    encoded_text + [pad_token_id] *
    (self.max_length - len(encoded_text))
    for encoded_text in self.encoded_texts
    ]

 def __getitem__(self, index):
  encoded = self.encoded_texts[index]
  label = self.data.iloc[index]["Label"]
  return (
    torch.tensor(encoded, dtype=torch.long),
    torch.tensor(label, dtype=torch.long)
    )

 def __len__(self):
  return len(self.data)

 def _longest_encoded_length(self):
  max_length = 0
  for encoded_text in self.encoded_texts:
    encoded_length = len(encoded_text)
    if encoded_length > max_length:
      max_length = encoded_length
  return max_length

In [12]:
train_dataset = SpamDataset(
 csv_file="train.csv",
 max_length=None,
 tokenizer=tokenizer
)

In [13]:
print(train_dataset.max_length)

120


pad the validation and test sets to match the length of the longest training sequence

In [14]:
val_dataset = SpamDataset(
 csv_file="validation.csv",
 max_length=train_dataset.max_length,
 tokenizer=tokenizer
)
test_dataset = SpamDataset(
 csv_file="test.csv",
 max_length=train_dataset.max_length,
 tokenizer=tokenizer
)

Creating Dataloader

In [15]:
from torch.utils.data import DataLoader
num_workers = 0
batch_size = 8
torch.manual_seed(123)
train_loader = DataLoader(
 dataset=train_dataset,
 batch_size=batch_size,
 shuffle=True,
 num_workers=num_workers,
 drop_last=True,
)
val_loader = DataLoader(
 dataset=val_dataset,
 batch_size=batch_size,
 num_workers=num_workers,
 drop_last=False,
)
test_loader = DataLoader(
 dataset=test_dataset,
 batch_size=batch_size,
 num_workers=num_workers,
 drop_last=False,
)

In [16]:
for input_batch, target_batch in train_loader:
 pass
print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions", target_batch.shape)

Input batch dimensions: torch.Size([8, 120])
Label batch dimensions torch.Size([8])


In [17]:
print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

130 training batches
19 validation batches
38 test batches
